In [ ]:
import sys

# note that these are intentionally not added to pyproject.toml
!{sys.executable} -m pip install notebook ipython-sql SQLAlchemy duckdb-engine

In [1]:
import duckdb
import pandas as pd
#import sqlalchemy
# No need to import duckdb_engine
#  SQLAlchemy will auto-detect the driver needed based on your connection string!

# Import ipython-sql Jupyter extension to create SQL cells
%load_ext sql

In [2]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [3]:
%sql duckdb:///data/f1.duckdb

In [4]:
%%sql
select * from information_schema.tables
WHERE table_schema = 'core'

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,None,core,d_circuits,BASE TABLE,None,None,None,None,None,YES,NO,None
1,None,core,d_constructors,BASE TABLE,None,None,None,None,None,YES,NO,None
2,None,core,f_qualifying,BASE TABLE,None,None,None,None,None,YES,NO,None
3,None,core,d_races,BASE TABLE,None,None,None,None,None,YES,NO,None
4,None,core,f_results,BASE TABLE,None,None,None,None,None,YES,NO,None
5,None,core,d_seasons,BASE TABLE,None,None,None,None,None,YES,NO,None
6,None,core,f_laps,BASE TABLE,None,None,None,None,None,YES,NO,None
7,None,core,d_drivers,BASE TABLE,None,None,None,None,None,YES,NO,None


In [9]:
%%sql
select * from information_schema.columns
WHERE table_name = 'f_qualifying'

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,None,core,f_qualifying,qualifying_id,1,None,YES,VARCHAR,None,None,...,None,None,None,None,None,None,None,None,None,None
1,None,core,f_qualifying,qualifying_date,2,None,YES,DATE,None,None,...,None,None,None,None,None,None,None,None,None,None
2,None,core,f_qualifying,race_id,3,None,YES,VARCHAR,None,None,...,None,None,None,None,None,None,None,None,None,None
3,None,core,f_qualifying,circuit_id,4,None,YES,VARCHAR,None,None,...,None,None,None,None,None,None,None,None,None,None
4,None,core,f_qualifying,driver_id,5,None,YES,VARCHAR,None,None,...,None,None,None,None,None,None,None,None,None,None
5,None,core,f_qualifying,constructor_id,6,None,YES,VARCHAR,None,None,...,None,None,None,None,None,None,None,None,None,None
6,None,core,f_qualifying,qualifying_position,7,None,YES,SMALLINT,None,None,...,None,None,None,None,None,None,None,None,None,None
7,None,core,f_qualifying,qualifying1_lap_time,8,None,YES,VARCHAR,None,None,...,None,None,None,None,None,None,None,None,None,None
8,None,core,f_qualifying,qualifying2_lap_time,9,None,YES,VARCHAR,None,None,...,None,None,None,None,None,None,None,None,None,None
9,None,core,f_qualifying,qualifying3_lap_time,10,None,YES,VARCHAR,None,None,...,None,None,None,None,None,None,None,None,None,None


In [15]:
%%sql
SELECT f.race_date
    , drac.race_season
    , drac.race_round
    , drac.race_name
    , dcir.circuit_country
    , ddri.driver_full_name
    , ddri.driver_permanent_number
    , dcon.constructor_name
    , f.result_position
    , f.result_points
    , f.fastest_lap
    , f.result_status
FROM core.f_results AS f
LEFT JOIN core.d_races AS drac ON drac.race_id = f.race_id
LEFT JOIN core.d_circuits AS dcir ON dcir.circuit_id = f.circuit_id
LEFT JOIN core.d_drivers AS ddri ON ddri.driver_id = f.driver_id
LEFT JOIN core.d_constructors AS dcon ON dcon.constructor_id = f.constructor_id
WHERE f.race_date BETWEEN '2022-01-01' AND '2022-12-31'
ORDER BY f.race_date DESC
    , f.result_position ASC

,race_date,race_season,race_round,race_name,circuit_country,driver_full_name,driver_permanent_number,constructor_name,result_position,result_points,fastest_lap,result_status
0,2022-10-30,2022,20,Mexico City Grand Prix,Mexico,Max Verstappen,33,Red Bull,1,25.0,False,Finished
1,2022-10-30,2022,20,Mexico City Grand Prix,Mexico,Lewis Hamilton,44,Mercedes,2,18.0,False,Finished
2,2022-10-30,2022,20,Mexico City Grand Prix,Mexico,Sergio Pérez,11,Red Bull,3,15.0,False,Finished
3,2022-10-30,2022,20,Mexico City Grand Prix,Mexico,George Russell,63,Mercedes,4,13.0,True,Finished
4,2022-10-30,2022,20,Mexico City Grand Prix,Mexico,Carlos Sainz,55,Ferrari,5,10.0,False,Finished
...,...,...,...,...,...,...,...,...,...,...,...,...
395,2022-03-20,2022,1,Bahrain Grand Prix,Bahrain,Nicholas Latifi,6,Williams,16,0.0,False,Finished
396,2022-03-20,2022,1,Bahrain Grand Prix,Bahrain,Nico Hülkenberg,27,Aston Martin,17,0.0,False,Finished
397,2022-03-20,2022,1,Bahrain Grand Prix,Bahrain,Sergio Pérez,11,Red Bull,18,0.0,False,Fuel pressure
398,2022-03-20,2022,1,Bahrain Grand Prix,Bahrain,Max Verstappen,33,Red Bull,19,0.0,False,Fuel pressure


In [14]:
%%sql
SELECT f.race_date
    , drac.race_season
    , drac.race_round
    , drac.race_name
    , dcir.circuit_country
    , ddri.driver_full_name
    , ddri.driver_permanent_number
    , f.lap_number
    , f.lap_position
    , f.lap_time
FROM core.f_laps AS f
LEFT JOIN core.d_races AS drac ON drac.race_id = f.race_id
LEFT JOIN core.d_circuits AS dcir ON dcir.circuit_id = f.circuit_id
LEFT JOIN core.d_drivers AS ddri ON ddri.driver_id = f.driver_id
WHERE f.race_date BETWEEN '2022-01-01' AND '2022-12-31'
ORDER BY f.race_date DESC
    , ddri.driver_full_name ASC
    , f.lap_number ASC

,race_date,race_season,race_round,race_name,circuit_country,driver_full_name,driver_permanent_number,lap_number,lap_position,lap_time
0,2022-10-30,2022,20,Mexico City Grand Prix,Mexico,Alexander Albon,23,1,19,1:36.903
1,2022-10-30,2022,20,Mexico City Grand Prix,Mexico,Alexander Albon,23,10,17,1:24.960
2,2022-10-30,2022,20,Mexico City Grand Prix,Mexico,Alexander Albon,23,11,17,1:25.159
3,2022-10-30,2022,20,Mexico City Grand Prix,Mexico,Alexander Albon,23,12,17,1:25.312
4,2022-10-30,2022,20,Mexico City Grand Prix,Mexico,Alexander Albon,23,13,17,1:26.206
...,...,...,...,...,...,...,...,...,...,...
21151,2022-03-20,2022,1,Bahrain Grand Prix,Bahrain,Yuki Tsunoda,22,57,8,1:38.891
21152,2022-03-20,2022,1,Bahrain Grand Prix,Bahrain,Yuki Tsunoda,22,6,11,1:40.611
21153,2022-03-20,2022,1,Bahrain Grand Prix,Bahrain,Yuki Tsunoda,22,7,11,1:40.754
21154,2022-03-20,2022,1,Bahrain Grand Prix,Bahrain,Yuki Tsunoda,22,8,11,1:41.351


In [13]:
%%sql
SELECT f.qualifying_date
    , drac.race_season
    , drac.race_round
    , drac.race_name
    , dcir.circuit_country
    , ddri.driver_full_name
    , ddri.driver_permanent_number
    , f.qualifying_position
    , f.qualifying1_lap_time
    , f.qualifying2_lap_time
    , f.qualifying3_lap_time
FROM core.f_qualifying AS f
LEFT JOIN core.d_races AS drac ON drac.race_id = f.race_id
LEFT JOIN core.d_circuits AS dcir ON dcir.circuit_id = f.circuit_id
LEFT JOIN core.d_drivers AS ddri ON ddri.driver_id = f.driver_id
WHERE f.qualifying_date BETWEEN '2022-01-01' AND '2022-12-31'
ORDER BY f.qualifying_date DESC
    , f.qualifying_position ASC

,qualifying_date,race_season,race_round,race_name,circuit_country,driver_full_name,driver_permanent_number,qualifying_position,qualifying1_lap_time,qualifying2_lap_time,qualifying3_lap_time
0,2022-11-13,2022,21,Brazilian Grand Prix,Brazil,Kevin Magnussen,20,1,1:13.954,1:11.410,1:11.674
1,2022-11-13,2022,21,Brazilian Grand Prix,Brazil,Max Verstappen,33,2,1:13.625,1:10.881,1:11.877
2,2022-11-13,2022,21,Brazilian Grand Prix,Brazil,George Russell,63,3,1:14.427,1:11.318,1:12.059
3,2022-11-13,2022,21,Brazilian Grand Prix,Brazil,Lando Norris,4,4,1:13.106,1:11.377,1:12.263
4,2022-11-13,2022,21,Brazilian Grand Prix,Brazil,Carlos Sainz,55,5,1:14.680,1:10.890,1:12.357
...,...,...,...,...,...,...,...,...,...,...,...
415,2022-03-20,2022,1,Bahrain Grand Prix,Bahrain,Yuki Tsunoda,22,16,1:32.750,None,None
416,2022-03-20,2022,1,Bahrain Grand Prix,Bahrain,Nico Hülkenberg,27,17,1:32.777,None,None
417,2022-03-20,2022,1,Bahrain Grand Prix,Bahrain,Daniel Ricciardo,3,18,1:32.945,None,None
418,2022-03-20,2022,1,Bahrain Grand Prix,Bahrain,Lance Stroll,18,19,1:33.032,None,None


In [17]:
%%sql
select *
FROM core.d_drivers 
WHERE most_recent_race_date >= '2022-01-01'
ORDER BY driver_second_name

,driver_id,driver_second_name,driver_first_name,driver_full_name,driver_code,driver_permanent_number,driver_date_of_birth,driver_nationality,driver_url,first_race_date,most_recent_race_date,number_of_races,lifetime_points,load_dts
0,albon,Albon,Alexander,Alexander Albon,ALB,23,1996-03-23,Thai,http://en.wikipedia.org/wiki/Alexander_Albon,2019-03-17,2022-10-30,57,201.0,2022-11-12 15:03:24.640
1,alonso,Alonso,Fernando,Fernando Alonso,ALO,14,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,2001-03-04,2022-10-30,356,2051.0,2022-11-12 15:03:24.640
2,bottas,Bottas,Valtteri,Valtteri Bottas,BOT,77,1989-08-28,Finnish,http://en.wikipedia.org/wiki/Valtteri_Bottas,2013-03-17,2022-10-30,199,1776.0,2022-11-12 15:03:24.640
3,gasly,Gasly,Pierre,Pierre Gasly,GAS,10,1996-02-07,French,http://en.wikipedia.org/wiki/Pierre_Gasly,2017-10-01,2022-10-30,106,332.0,2022-11-12 15:03:24.640
4,hamilton,Hamilton,Lewis,Lewis Hamilton,HAM,44,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,2007-03-18,2022-10-30,308,4378.5,2022-11-12 15:03:24.640
5,hulkenberg,Hülkenberg,Nico,Nico Hülkenberg,HUL,27,1987-08-19,German,http://en.wikipedia.org/wiki/Nico_H%C3%BClkenberg,2010-03-14,2022-03-27,184,521.0,2022-11-12 15:03:24.640
6,latifi,Latifi,Nicholas,Nicholas Latifi,LAT,6,1995-06-29,Canadian,http://en.wikipedia.org/wiki/Nicholas_Latifi,2020-07-05,2022-10-30,59,9.0,2022-11-12 15:03:24.640
7,leclerc,Leclerc,Charles,Charles Leclerc,LEC,16,1997-10-16,Monegasque,http://en.wikipedia.org/wiki/Charles_Leclerc,2018-03-25,2022-10-30,101,821.0,2022-11-12 15:03:24.640
8,kevin_magnussen,Magnussen,Kevin,Kevin Magnussen,MAG,20,1992-10-05,Danish,http://en.wikipedia.org/wiki/Kevin_Magnussen,2014-03-16,2022-10-30,140,179.0,2022-11-12 15:03:24.640
9,norris,Norris,Lando,Lando Norris,NOR,4,1999-11-13,British,http://en.wikipedia.org/wiki/Lando_Norris,2019-03-17,2022-10-30,80,413.0,2022-11-12 15:03:24.640
